In [1]:
import pathlib
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, LineString, Point
import math
import warnings
warnings.filterwarnings(action='ignore')
#import descartes
import folium
from tqdm.notebook import tqdm

In [77]:
import sklearn.cluster
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
%cd E:\yujin\INHA\data

E:\yujin\INHA\data


In [8]:
건물분포도 = gpd.read_file("17.광양시_건물분포도(연면적)_격자(100X100).geojson")
건물분포도

,gid,val,geometry
0,라라023745,NaN,"MULTIPOLYGON (((127.52523 35.06557, 127.52523 ..."
1,라라023746,NaN,"MULTIPOLYGON (((127.52523 35.06647, 127.52523 ..."
2,라라023747,NaN,"MULTIPOLYGON (((127.52523 35.06737, 127.52523 ..."
3,라라023748,NaN,"MULTIPOLYGON (((127.52523 35.06828, 127.52523 ..."
4,라라023749,NaN,"MULTIPOLYGON (((127.52523 35.06918, 127.52523 ..."
...,...,...,...
47027,라라261688,NaN,"MULTIPOLYGON (((127.78607 35.01384, 127.78607 ..."
47028,라라261689,NaN,"MULTIPOLYGON (((127.78607 35.01474, 127.78608 ..."
47029,라라261690,NaN,"MULTIPOLYGON (((127.78608 35.01564, 127.78608 ..."
47030,라라261691,NaN,"MULTIPOLYGON (((127.78608 35.01654, 127.78608 ..."


In [14]:
건물 = gpd.read_file("15.광양시_건물정보.geojson")
건물

,고유번호,법정동코드,법정동명,특수지구분코드,특수지구분명,지번,건물명,건물동명,건물주부구분코드,건물주부구분명,...,건축물구조코드,건축물구조명,주요용도코드,주요용도명,세부용도코드,세부용도명,건물높이,지상층수,지하층수,geometry
0,4623010100100160001,4623010100,전라남도 광양시 황금동,1,일반,16-1,None,None,0,주건축물,...,51,일반목구조,01000,단독주택,01001,단독주택,0.00,1.0,0.0,"MULTIPOLYGON (((127.64656 34.91358, 127.64671 ..."
1,4623010100100630000,4623010100,전라남도 광양시 황금동,1,일반,63,None,제1동,0,주건축물,...,21,철근콘크리트구조,04000,제2종근린생활시설,04402,사무소,7.90,2.0,0.0,"MULTIPOLYGON (((127.65097 34.91113, 127.65117 ..."
2,4623010100100630000,4623010100,전라남도 광양시 황금동,1,일반,63,None,제2동,0,주건축물,...,31,일반철골구조,18000,창고시설,18001,창고,11.70,1.0,0.0,"MULTIPOLYGON (((127.65061 34.91137, 127.65054 ..."
3,4623010100100630000,4623010100,전라남도 광양시 황금동,1,일반,63,None,제3동,0,주건축물,...,31,일반철골구조,18000,창고시설,18001,창고,7.00,1.0,0.0,"MULTIPOLYGON (((127.65096 34.91113, 127.65096 ..."
4,4623010100100780004,4623010100,전라남도 광양시 황금동,1,일반,78-4,황금교회,None,0,주건축물,...,32,경량철골구조,04000,제2종근린생활시설,04201,교회,4.20,1.0,0.0,"MULTIPOLYGON (((127.65002 34.91084, 127.65010 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23551,4623036025102480000,4623036025,전라남도 광양시 다압면 하천리,1,일반,248,None,None,0,주건축물,...,21,철근콘크리트구조,01000,단독주택,01001,단독주택,4.65,1.0,0.0,"MULTIPOLYGON (((127.61374 35.16463, 127.61375 ..."
23552,4623036025102640000,4623036025,전라남도 광양시 다압면 하천리,1,일반,264,None,주건축물제1동,0,주건축물,...,51,일반목구조,01000,단독주택,01001,단독주택,6.00,2.0,0.0,"MULTIPOLYGON (((127.61128 35.16372, 127.61146 ..."
23553,4623036025103110002,4623036025,전라남도 광양시 다압면 하천리,1,일반,311-2,None,None,0,주건축물,...,11,벽돌구조,01000,단독주택,01001,단독주택,0.00,1.0,0.0,"MULTIPOLYGON (((127.61393 35.16402, 127.61399 ..."
23554,4623036025103420000,4623036025,전라남도 광양시 다압면 하천리,1,일반,342,None,None,0,주건축물,...,32,경량철골구조,18000,창고시설,18001,창고,3.50,1.0,0.0,"MULTIPOLYGON (((127.61411 35.16438, 127.61415 ..."


In [12]:
table_raw = pd.read_csv("테이블.csv", index_col=0)
table_raw

,area,주차면수,lon,lat,geometry,category,소유주체,급속/완속,완속충전기_설치가능여부,인구분포,자동차등록현황,administration,전기차증가량
0,4189.00,161.0,127.586883,34.970324,POINT (127.5868828 34.97032367),주차장,1,NaN,1,0.241079,0.045100,광양읍,1.262407
1,2968.00,125.0,127.695428,34.936252,POINT (127.6954279 34.93625189999999),주차장,1,NaN,1,0.591851,0.648245,중마동,-0.162234
2,2836.00,89.0,127.584586,34.975637,POINT (127.5845858 34.97563677),주차장,1,NaN,1,0.039610,-0.130978,광양읍,1.262407
3,1980.00,52.0,127.580683,34.970130,POINT (127.5806828 34.97013011),주차장,1,NaN,1,0.266335,0.044476,광양읍,1.262407
4,1970.00,64.0,127.580707,34.972523,POINT (127.5807067 34.97252282),주차장,1,NaN,1,0.199328,-0.024593,광양읍,1.262407
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1205,79.95,0.0,127.768449,34.985126,POINT (127.7684494 34.98512629),휴게소,2,NaN,0,-0.679153,-0.321148,진월면,-1.171355
1206,48.00,0.0,127.768300,34.984707,POINT (127.7682997 34.98470653),휴게소,2,NaN,0,-0.681891,-0.323314,진월면,-1.171355
1207,77.49,0.0,127.769068,34.985767,POINT (127.7690681 34.98576671),휴게소,2,NaN,0,-0.647752,-0.292788,진월면,-1.171355
1208,54.00,0.0,127.769183,34.985841,POINT (127.7691831 34.98584074),휴게소,1,NaN,0,-0.643474,-0.287183,진월면,-1.171355


In [13]:
table = table_raw[['lon', 'lat', 'category']]
table

,lon,lat,category
0,127.586883,34.970324,주차장
1,127.695428,34.936252,주차장
2,127.584586,34.975637,주차장
3,127.580683,34.970130,주차장
4,127.580707,34.972523,주차장
...,...,...,...
1205,127.768449,34.985126,휴게소
1206,127.768300,34.984707,휴게소
1207,127.769068,34.985767,휴게소
1208,127.769183,34.985841,휴게소


In [18]:
population = gpd.read_file("08.광양시_격자별인구현황(100X100).geojson")

In [20]:
distance = 60*1000/6

scale = sum([
    *(population['geometry'].bounds['maxx'] - population['geometry'].bounds['minx']),
    *(population['geometry'].bounds['maxy'] - population['geometry'].bounds['miny']),
]) / (2 * len(population)) / 100

In [48]:
세부용도명 = 건물['세부용도명'].unique().tolist()

건물_경위도 = np.stack([건물['geometry'].centroid.x, 건물['geometry'].centroid.y], axis=1).reshape([1,-1,2])
건물_속성 = 건물.loc[:, '세부용도명'].apply(세부용도명.index).values.reshape([1,-1])
lon_lat = np.stack([table.lon, table.lat], axis=1).reshape([-1,1,2])


건물_조건 = np.sum((lon_lat-건물_경위도)**2, axis=2)**0.5 < 399.3*2 * scale
건물_데이터 = np.zeros([len(세부용도명), len(lon_lat)])
for i, x in enumerate(건물_속성 *건물_조건):
    x = set(x)
    건물_데이터[:,i] = pd.Series(range(len(세부용도명))).apply(x.__contains__).values

In [49]:
카테고리 = table['category']
데이터 = pd.DataFrame(columns=['카테고리',*세부용도명],
                  data = np.stack([카테고리, *건물_데이터], axis=1))

In [50]:
데이터

,카테고리,단독주택,사무소,창고,교회,일반공장,노인복지시설,축사,마을회관,기타창고시설,...,고압가스저장소,대피소,직업훈련소,학교,액화석유가스판매소,가축용창고,사당,청소년수련원,생활숙박시설,양잠
0,주차장,1,1,1,1,1,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,주차장,1,1,1,1,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,주차장,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,주차장,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,주차장,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1205,휴게소,1,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1206,휴게소,1,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1207,휴게소,1,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1208,휴게소,1,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
데이터 = 데이터.iloc[:,1:]
데이터

,단독주택,사무소,창고,교회,일반공장,노인복지시설,축사,마을회관,기타창고시설,제조업소,...,고압가스저장소,대피소,직업훈련소,학교,액화석유가스판매소,가축용창고,사당,청소년수련원,생활숙박시설,양잠
0,1,1,1,1,1,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,1,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1205,1,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1206,1,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1207,1,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1208,1,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [53]:
table2 = table_raw[['category','인구분포', '자동차등록현황','전기차증가량']]
train_table = pd.concat([table2, 데이터], axis=1)
train_table

,category,인구분포,자동차등록현황,전기차증가량,단독주택,사무소,창고,교회,일반공장,노인복지시설,...,고압가스저장소,대피소,직업훈련소,학교,액화석유가스판매소,가축용창고,사당,청소년수련원,생활숙박시설,양잠
0,주차장,0.241079,0.045100,1.262407,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,주차장,0.591851,0.648245,-0.162234,1,1,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2,주차장,0.039610,-0.130978,1.262407,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,주차장,0.266335,0.044476,1.262407,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,주차장,0.199328,-0.024593,1.262407,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1205,휴게소,-0.679153,-0.321148,-1.171355,1,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1206,휴게소,-0.681891,-0.323314,-1.171355,1,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1207,휴게소,-0.647752,-0.292788,-1.171355,1,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1208,휴게소,-0.643474,-0.287183,-1.171355,1,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [54]:
train_table['충전기'] = 0
for i in range(len(train_table)):
    if train_table['category'][i] == '충전기':
        train_table['충전기'][i] = 1
        
train_table.drop(['category'], axis=1, inplace=True )
train_table

,인구분포,자동차등록현황,전기차증가량,단독주택,사무소,창고,교회,일반공장,노인복지시설,축사,...,대피소,직업훈련소,학교,액화석유가스판매소,가축용창고,사당,청소년수련원,생활숙박시설,양잠,충전기
0,0.241079,0.045100,1.262407,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0.591851,0.648245,-0.162234,1,1,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0.039610,-0.130978,1.262407,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,0.266335,0.044476,1.262407,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,0.199328,-0.024593,1.262407,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1205,-0.679153,-0.321148,-1.171355,1,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1206,-0.681891,-0.323314,-1.171355,1,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1207,-0.647752,-0.292788,-1.171355,1,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1208,-0.643474,-0.287183,-1.171355,1,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [57]:
x_train, x_test, y_train, y_test = train_test_split(train_table.iloc[:,:232], train_table['충전기'], test_size=0.3, random_state=5)
y_train = y_train.values.ravel()

In [59]:
model = LogisticRegression(solver='saga', max_iter=10000)

In [60]:
model.fit(x_train, y_train)

LogisticRegression(max_iter=10000, solver='saga')

In [61]:
model.predict(x_test)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [63]:
model.score(x_test, y_test)

0.9944903581267218

# 진짜 모델

In [64]:
train_table

,인구분포,자동차등록현황,전기차증가량,단독주택,사무소,창고,교회,일반공장,노인복지시설,축사,...,대피소,직업훈련소,학교,액화석유가스판매소,가축용창고,사당,청소년수련원,생활숙박시설,양잠,충전기
0,0.241079,0.045100,1.262407,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0.591851,0.648245,-0.162234,1,1,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0.039610,-0.130978,1.262407,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,0.266335,0.044476,1.262407,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,0.199328,-0.024593,1.262407,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1205,-0.679153,-0.321148,-1.171355,1,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1206,-0.681891,-0.323314,-1.171355,1,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1207,-0.647752,-0.292788,-1.171355,1,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1208,-0.643474,-0.287183,-1.171355,1,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [76]:
table2 = table_raw[['lon','lat','인구분포','자동차등록현황','전기차증가량','category']]
table3 = table2[table2['category'] != '충전기']
table3.drop(['category'], axis=1, inplace=True)
table3.reset_index(drop=True, inplace=True)
table3

,lon,lat,인구분포,자동차등록현황,전기차증가량
0,127.586883,34.970324,0.241079,0.045100,1.262407
1,127.695428,34.936252,0.591851,0.648245,-0.162234
2,127.584586,34.975637,0.039610,-0.130978,1.262407
3,127.580683,34.970130,0.266335,0.044476,1.262407
4,127.580707,34.972523,0.199328,-0.024593,1.262407
...,...,...,...,...,...
1161,127.768449,34.985126,-0.679153,-0.321148,-1.171355
1162,127.768300,34.984707,-0.681891,-0.323314,-1.171355
1163,127.769068,34.985767,-0.647752,-0.292788,-1.171355
1164,127.769183,34.985841,-0.643474,-0.287183,-1.171355


In [79]:
weight = table3[['인구분포','자동차등록현황','전기차증가량']]

In [81]:
kmeans = sklearn.cluster.KMeans(n_clusters = 40)
kmeans.fit(X = weight)

KMeans(n_clusters=40)

In [83]:
predict = pd.DataFrame(kmeans.predict(weight))
predict.columns=['predict']

In [84]:
predict

,predict
0,30
1,16
2,2
3,30
4,2
...,...
1161,36
1162,36
1163,36
1164,36
